<a href="https://colab.research.google.com/github/dsahoo17/Flair_NER/blob/master/cellphone_reviews_TBSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#URL used: https://www.kaggle.com/datasets/grikomsn/amazon-cell-phones-reviews

project_directory = "phone_review"

from os.path import exists
import os
os.chdir("/content")

if not exists(project_directory):
  os.mkdir(project_directory)

os.chdir("/content/"+project_directory)

#upload file to phone_review

In [ ]:
os.getcwd()

In [ ]:
import shutil
shutil.unpack_archive("cellphone_reviews.zip", "./")

In [ ]:
#upload kaggle api key
# ! cp ../kaggle.json /root/.kaggle
# !kaggle datasets download -d grikomsn/amazon-cell-phones-reviews

In [ ]:
### dependency ###
import nltk
nltk.download('stopwords')
!pip install langdetect
!pip install aspect_based_sentiment_analysis

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!pip install spacy
!spacy download en_core_web_md

In [ ]:
#Read Cellphone reviews data
import pandas as pd
items_df = pd.read_csv(r"20191226-items.csv",encoding='utf-8')
reviews_df = pd.read_csv(r"20191226-reviews.csv",encoding='utf-8')

In [ ]:
reviews_sample_df = reviews_df[["asin","body"]]
items_sample_df = items_df[["asin","brand","title"]]
#Merge based on common column "asin"
data_df = pd.merge(reviews_sample_df, items_sample_df, on="asin")

In [ ]:
data = data_df.copy(deep=True)

In [ ]:
data.head(2)

In [ ]:
from nltk.corpus import stopwords
en_stopwords = set(stopwords.words('english'))

ignore_stopwords = ['no','not',"doesn't","didn't","couldn't","isn't"]
for iw in ignore_stopwords:
    en_stopwords.remove(iw)

def pre_process(text):
    text = str(text).lower()
    sttt = []
    for p in '!"#%&\'()*+\,-./:;<=>?@[\\]^_`{|}~':
        text = text.replace(p,"")
    for word in text.split(" "):
        if word not in en_stopwords or "n't" in word:
            sttt.append(word)
    return (" ").join(sttt)

data["cleaned_body"] = data["body"].apply(pre_process).str.replace(r' +',' ',regex=True)

In [ ]:
from langdetect import detect
# data["language"] = data["body"].apply(detect)
language_lst = []
for index,row in data.iterrows():
    if index%2000 == 0:
        print("index:",index)
    try:
        language = detect(row["cleaned_body"])
    except:
        language = "language_not_detected"
        # print("Error! Language not detected in:", row["cleaned_body"])
    language_lst.append(language)
data["language"]  = language_lst

In [ ]:
data = data[data["language"]=='en']
data = data[["asin","body","cleaned_body"]]

In [ ]:
import aspect_based_sentiment_analysis as absa

In [ ]:
nlp = absa.load()

In [ ]:
data["tokenized_body"] = data['cleaned_body'].apply(nltk.word_tokenize)

In [ ]:
data.head(10)

In [ ]:
import spacy
spacy_nlp = spacy.load("en_core_web_md")
def spacy_pre_process(txt):
    req_tag = ['NN']
    extracted_nouns = []
    tokenized_body = []
    try:
        doc = spacy_nlp(txt)
        for token in doc:
            tokenized_body.append(token.lemma_)
            if token.tag_ in req_tag:
                extracted_nouns.append(token.lemma_)
        return pd.Series([list(set(extracted_nouns)),tokenized_body])
    except Exception as e:
        print(e)
        return pd.Series([list(set(extracted_nouns)),tokenized_body])

In [ ]:
data[["Spacy_Nouns","tokenized_body"]] =  data["cleaned_body"].apply(spacy_pre_process)


In [ ]:
data.head(10)

In [ ]:
all_noun_list = []
for idx,row in data.Spacy_Nouns.iteritems():
    for r in row:
        all_noun_list.append(r)

In [ ]:
from collections import Counter
counts = Counter(all_noun_list)
freq_list = sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
freq_list[0:20]

In [ ]:
aspects = ["phone","battery","screen","camera","price","charger"]

In [ ]:
data.to_csv("./data_full_nouns.csv")

In [ ]:
data.head(5)

In [ ]:

import gensim
from gensim.models import word2vec

# Load the binary model
model = word2vec.Word2Vec(data['tokenized_body'],workers=10,
                          size=300,min_count=2,window=10)

In [ ]:
model.most_similar("bluetooth")

In [ ]:
model.most_similar("smartphone")

In [ ]:
sc = "camera is good but battery is draining so fast."
sc = "I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing."
aspects = ["phone","battery","screen","camera","price","charger"]

def calculate_aspect(txt):
  TH = 0.6
  result = {}
  # for a in aspects:
  #   result[a]="neutral"
  txt = txt.lower()
  # print(result)
  for items in list(nlp(txt, aspects=aspects).subtasks.items()):
      # print(items[0],items[1].sentiment.name, items[1].scores)
      if max(items[1].scores) > TH:
          result[str(items[0])] = items[1].sentiment.name
  return result

print(calculate_aspect(sc))


In [ ]:
data_bkp = data.sample(frac=0.0001)
data_bkp["aspect"] = data_bkp["cleaned_body"].apply(calculate_aspect)

In [ ]:
data_bkp.head(100)

In [ ]:
data_bkp.body.iloc[0]

In [ ]:
import spacy
sp = spacy.load("en_core_web_md")
from textblob import TextBlob
  
# Creating a list of positive and negative sentences.
mixed_sen = [
  "camera is good but battery is draining so fast."
]
  
# An empty list for obtaining the extracted aspects
# from sentences. 
ext_aspects = []
  
# Performing Aspect Extraction
for sen in mixed_sen:
  important = sp(sen)
  descriptive_item = ''
  target = ''
  for token in important:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      added_terms = ''
      for mini_token in token.children:
        if mini_token.pos_ != 'ADV':
          continue
        added_terms += mini_token.text + ' '
      descriptive_item = added_terms + token.text
  ext_aspects.append({'aspect': target,
    'description': descriptive_item})
  
print("ASPECT EXTRACTION\n")
print(ext_aspects)
  
  
for aspect in ext_aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
  
print("\n")
print("SENTIMENT ASSOCIATION\n")
print(ext_aspects)